For demo purposes.

# Table of Content

1. [Data_Dictionary](#data-dictionary)
2. [Demo](#demo)

# Data Dictionary

|Feature|Type|Description|
|---|---|---|
|Product_Code|*string*|The product name encoded|
|Warehouse|*string*|Warehouse ID|
|Product_Category|*string*|Product category of each product|
|Date|*string*|The date customer needs the product|
|Order_Demand|*string*|Order quantity|

In [11]:
# Import libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from numpy import array
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [12]:
# Read csv file and display shape.
df = pd.read_csv('../Data_Historical/df2.csv').iloc[:,1:]

# Get list of product codes.
products = df['Product_Code'].unique()
df.shape

(924272, 5)

In [13]:
# choose a number of time steps and features
n_steps = 1
n_features = 1

In [14]:
# Function to clean and resample order demand.
def get_product(data,product_code):
    df_init = data[data['Product_Code']==product_code]
    df_init['Date'] = pd.to_datetime(df_init['Date'])
    df_init = df_init.set_index('Date')
    df_out = pd.DataFrame(df_init['Order_Demand'].resample('W').sum())
    df_out.rename(columns={'Order_Demand':'demand'+'_'+product_code[-4:]}, inplace=True)
    return df_out

In [15]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
 X, y = list(), list()
 
 for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
        break
    
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
 return array(X), array(y)

In [16]:
# Function to apply standard scaling to input.
def scale(X_train): 
 # fit scaler
 scaler = StandardScaler()
 # Fit & transform train
 X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
 return scaler, X_train

In [17]:
# Function to reshape and scale the previous week's sales.
def reshape_scale(y0, scaler):
    # Reshape to (1,1,1)
    wk0 = y0[-1:].reshape((1,1,n_features))
    # Transform to standard scaling
    wk0 = scaler.transform(wk0.reshape(-1, wk0.shape[-1])).reshape(wk0.shape)
    return wk0

In [18]:
# LSTM Model.
def fit_lstm (X, y):
    model = Sequential()
    model.add(LSTM(64, activation='relu', return_sequences=True,  input_shape=(n_steps, n_features))) 
    model.add(LSTM(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=.001), loss='mse')
    lstm = model.fit(X, y, epochs=100, verbose=0, batch_size=32)
    return model, lstm

In [19]:
# Function to run the model and predictions
def run(df, product):
    n_steps = 1
    n_features = 1
    preds = list()   
    # Create dataframe of Product
    prod = get_product(df, product)
    # define input sequence
    raw_seq = prod.iloc[:,0]
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    # Standard scaling X
    scaler, X_scaled = scale(X)
    model, lstm = fit_lstm(X_scaled, y)
    wk0 = reshape_scale(y[-1:], scaler)
    # Predict Week 1
    predwk1 = model.predict(wk0)
    preds.append(predwk1)
    # Predict Week 2
    predwk2 = model.predict(reshape_scale(predwk1, scaler))
    preds.append(predwk2)
    # Predict Week 3
    predwk3 = model.predict(reshape_scale(predwk2, scaler))
    preds.append(predwk3)
    # Predict Week 4
    predwk4 = model.predict(reshape_scale(predwk3, scaler))
    preds.append(predwk4)
    # Reshape predictions to 2 D
    reshaped = np.reshape(preds, (4,))
    return reshaped

# Demo

In [20]:
target = 'Product_0614'
predictions = run(df, target)
df_prod = pd.DataFrame({target:predictions})
df_prod = df_prod.astype(int)
df_prod

C:\Users\Zeonice\AppData\Local\Temp\ipykernel_14328\184514828.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init['Date'] = pd.to_datetime(df_init['Date'])


1/1 [==============================] - 0s 28ms/step


,Product_0614
0,46
1,37
2,38
3,38
